## Latent Dirichlet Allocation (LDiA)

Latent Sematic Analysis (LSA) should be the first choice for most topic modelling, semantic search, or content-based recommendation engines.
<br>
Given the maths used for LSA is efficient and somewhat easy to follow, given that it produces a linear transformation that can be applied to new batches of natural language without training and little loss in accuracy. However, LDiA can give slightly better results in some situations. 

LDiA performs many things accomplished by topic models with LSA (and SVD under the hood), though unlike LSA, LDiA assumes a Dirichlet distribution of word frequencies. It's more precise about the statistics of allocating words to topics than the linear math of LSA.


LDiA creates a semantic vector space model (as with topic vectors) using a topic mix process that for any given document, such topics can be determined by the word mixtures in each topic by which topic those words were assigned to.
<br>
In some ways, this makes an LDiA topic model easier to understand, as words assigned to topics and topics assigned to documents tend to make more sense than for LSA.
<br>
LDiA assumes that each document is a mixture (linear combination) of some arbitrary number of topics that one can select when beginning to train the LDiA model. LDiA also assumes that each topic can be represented by a distribution of words (term frequencies). 
<br>
The probability/weight for each of these topics within a document, along with the probability of a word being assigned to a topic, is assumed to begin with a Dirichlet probability distribution (*prior* in statistics), where the algorithm derives from.

### LDiA idea

Researchers like Blei and Ng proposed an idea by imagining how a machine that could only roll dice i.e. generate random numbers, could also write the documents in a corpus one may want to analyse. Given that one works with Bag of words during this process, it cuts out the part about sequencing such words together to make sense, to write a real document.
<br>
In this way, they modeled the statistics for the mix of words that would become a part of a particular BOW for each document.

They envisioned a machine that only had two choices to make to process generating the mix of words for a specific document. The two roles of 'dice' represent:
<br>
1. The number of words to generate for the document (Poisson distribution)
<br>
2. The number of topics to mix together for the document (Dirichlet distribution)

After these two numbers, the difficult task is to choose the words for the document.
<br>
The imaginary BOW generating machine iterates over those topics and randomly chooses words appropriate to that topic until it hits the number of words that it had decided the document should contain in step 1.
<br.
Deciding the probabilities of those words for topics — the appropriateness of words for each topic— is the hard part. But once that has been determined, your 'bot' just looks up the probabilities for the words for each topic from a matrix of term-topic probabilities.




So all this machine needs is a single parameter for that Poisson distribution (in the dice roll from step 1) that tells it what the 'average' document length should be, and a couple more parameters to define that Dirichlet distribution that sets up the number of topics.
<br>
Then your document generation algorithm needs a term-topic matrix of all the words and topics it likes to use, its vocabulary. And it needs a mix of topics that it likes to 'talk' about.

We can make this process more convenient in our use case, reversing the document generation (writing) problem back around to the original problem of estimating the topics and words from existing documents.
<br>
Initially, we need to measure or compute those parameters about words and topics for the first two steps.
<br>
Then we need to compute the term-topic matrix from a collection of documents. This is ultimately the process of LDiA.

The parameters for steps can be computed by analysing the statistics of the documents in a corpus.
<br>
For instance, to solve step 1, one could calculate the mean number of words (n-grams) in all the BOW for the documents in a specified corpus.
<br> 
An implementation in python may look like this:

In [3]:
import pandas as pd 
from nltk.tokenize import casual_tokenize
from nlpia.data.loaders import get_data
pd.options.display.width = 120 
sms = get_data('sms-spam')

In [4]:
total_corpus_len = 0 
for document_text in sms.text:
    total_corpus_len += len(casual_tokenize(document_text))

In [5]:
mean_document_len = total_corpus_len/len(sms)
round(mean_document_len, 2)

21.35

These document statistics are usually computed directly from the BOWs.
<br>
We need to make sure that we're counting the tokenized and vectorized text words with the specified documents.
<br>
It's also key to maintain a level of text normalization, particularly any stop word filtering, case folding etc. before counting up the unique terms.
<br>
This ensures that such count includes all the words in our BOW vector vocabulary (all the the n-grams we counting), but only those words that our BOW usa (i.e. not stop words)


The second parameter we need to specify for an LDiA model is the **number of topics** which can be tedious.
<br>
This is the case because the number of topics in a particular set of documents can't be measured directly until after we've assigned words to such topics. 
<br>
Clustering algorithms like ***K-means*** and ***KNN***, we would also need to pass the ***k*** parameter ahead of time.
<br>
 We can guess the number of topics (analogous to the ***k*** in k-means, which is the number of clusters) and then check to see if that works for one's set of documents.
<br>
Once specified to the LDA model how many topics to look for, it will find the mix of words to put in each topic to optimise its objective function.

We can optimise this 'hyperparameter' (***k***, the number of topics) by adjusting it until it works for our application.
<br>
We can automate this optimisation if we can measure something about the quality of the LDiA language model for representing the meaning of the documents.
<br>
A 'cost function' to assess the optimisation for this model is how well/poorly the LDiA model performs in classification or regression problems such as sentiment analysis, document keyword tagging or topic analysis. We only need some labeled documents to test our topic model or classifier on.

### LDiA topic model for SMS messages 

The topics produced by LDiA tend to be more understandable and interpretable to humans.
<br>
This is the case as words that frequently occur together are assigned the same topic(s), which is expected by most humans. 
<br>
Whereas LSA (PCA) tries to keep things spread apart to begin with, LDiA tries to keep things close together that started out close together.


Comparing LSA and LDiA, the math optimises for different things - based on the idea that the LDiA optimiser has a different objective function where it will reach a different objective.
<br>
To keep close high-dimensional vectors close together in the lower-dimensional space, LDiA has to twist and contort the space (and the vectors) in nonlinear ways.

To experiment with an LDiA topic model, let's see how it works for a dataset of a few thousand SMS messages, labelled for spaminess.
<br>
Initially, we compute the word vector transformation and then some topic vectors for SMS message (document). We assume the use of 16 topics (components) to classify the spaminess of messages.
<br>
Keeping the number of topics (dimensions) low can help reduce the overfitting problem.

Given LDiA works primarily with BOW count vectors rather than TF-IDF vectors - a workflow to compute BOW vectors in `sklearn` looks like this:

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
np.random.seed(42) # replicable results 

In [12]:
counter = CountVectorizer(tokenizer=casual_tokenize)
bow_vector = counter.fit_transform(raw_documents=sms.text)

In [15]:
bow_docs = pd.DataFrame(bow_vector.todense(), index=sms.index)
bow_docs.head() # test vector transformation 

,0,1,2,3,4,5,6,7,8,9,...,9222,9223,9224,9225,9226,9227,9228,9229,9230,9231
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
columns_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(), counter.vocabulary_.keys())))
bow_docs.columns = terms